In [1]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset


c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make Data

In [2]:
proper_review = glob.glob('./data/review_img/*/O/*')
proper_review = sorted(proper_review)
len(proper_review)

42526

In [3]:
unfit_review = glob.glob('./data/review_img/*/X/*')
unfit_review = sorted(unfit_review)
len(unfit_review)

3896

# Custom dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self,review_img_path,transform=None):
        self.review_img_path = review_img_path
        self.transform = transform
        
    def __getitem__(self,index):
        image = cv2.imread(self.review_img_path[index])
        image = letterbox(image, 960, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        return image
    
    def __len__(self):
        return len(self.review_img_path)

In [5]:
dataset = CustomDataset(np.array(unfit_review))
img_loader = DataLoader(dataset, batch_size = 1, shuffle=False, num_workers=0)

# Set model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weigths['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

In [7]:
def prdict(model, image_lodaer):
    # model.half().to(device)
    model.eval()
    
    proper = 1
    unproper = 0

    flag = True
    
    with torch.no_grad():
        for img in image_lodaer:
            flag = True
            img = img[0]
            
            img = img.half().to(device)
            output, _ = model(img)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
            
            if output.size == 0:
                unproper += 1
                continue
            
            for landmark_presence in output[0][15::3]:
                if landmark_presence < 0.1:
                    flag = False
                    break
                
            if flag:
                proper += 1
            else:
                unproper += 1

            print((unproper/(unproper + proper))*100)

In [8]:
prdict(model,img_loader )

c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


50.0
66.66666666666666
75.0
80.0
83.33333333333334
85.71428571428571
75.0
77.77777777777779
80.0
72.72727272727273
75.0
76.92307692307693
78.57142857142857
80.0
81.25
83.33333333333334
84.21052631578947
85.0
85.71428571428571
86.36363636363636
82.6086956521739
79.16666666666666
76.0
76.92307692307693
77.77777777777779
78.57142857142857
79.3103448275862
80.0
80.64516129032258
81.25
81.81818181818183
82.35294117647058
82.85714285714286
80.55555555555556
81.08108108108108
81.57894736842105
82.05128205128204
82.5
82.92682926829268
83.33333333333334
83.72093023255815
84.0909090909091
84.44444444444444
84.78260869565217
85.1063829787234
85.41666666666666
85.71428571428571
86.0
86.27450980392157
86.53846153846155
86.79245283018868
87.03703703703704
87.27272727272727
87.5
87.71929824561403
87.93103448275862
88.13559322033898
88.33333333333333
88.52459016393442
88.70967741935483
88.88888888888889
89.0625
89.23076923076924
89.39393939393939
89.55223880597015
89.70588235294117
89.85507246376811
9